# Generating new CDF Files
Combining data from multiple files into a single .cdf file, with both original values and calculated values

In [2]:
from scipy.io import netcdf
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import math
import os,sys
import matplotlib.style

import matplotlib
import netCDF4

matplotlib.style.use('classic')
import time
from datetime import datetime

In [3]:
import glob,os,sys
os.chdir('C:/Users/yashg/Documents/Cloud_Data_Files')
def read_files(extensions,location):
    l=[]
    for types in extensions:
        l.append(glob.glob(f'./{location}/*{types}'))
    l=[val for sublist in l for val in sublist]
    return l

k=read_files(['.cdf','.nc'],'KAZRARSCL')
m=read_files(['.cdf','.nc'],'Microbase')
r=read_files(['.cdf','.nc'],'Raman Lidar')
s=read_files(['.cdf','.nc'],'Surface')
e=read_files(['.cdf','.nc'],'Ext')
mp=read_files(['.cdf','.nc'],'Micropulse')
mwr=read_files(['.cdf','.nc'],'mwrret')
mic=read_files(['.cdf','.nc'],'ceilometer')
master=k+m+r+s+e+mp+mwr+mic

def date_files(date,master=master):
    f=[]
    for file in master:
        if date in file:
            f.append(file)
    return f

def generate_cdfs(date):
    l=[]
    f=date_files(date)
    for file in f:
        l.append(Dataset(file))
    print('Output has 6 files')
    print('File order is\t1.KAZRARSCL\t2.Microbase\t3.Raman Lidar\t4. Surface\t5. Ext\t6.Micropulse\t 7.Mwrret\t 8. Ceilometer')
    return l

In [4]:
def st_clustering(t1,h1,t2,h2):
    t_clust=[]
    h_clust=[]
    output_resolution=(len(t1),len(h1))
    data_resolution=(len(t2),len(h2))
    print('--------Start of clustering cycle--------')
    print(f'Changing time resolution from {len(t2)}->{len(t1)}')
    print(f'Changing height resolution from {len(h2)}->{len(h1)}')
    if len(t1)>len(t2):
        print('You are increasing time resolution')
        for i in t1:
            x=np.argmin(np.abs(i-t2))
            t_clust.append(x)
    else:
        print('You are decreasing time resolution')
        for i in range(len(t1)):
            if i!=len(t1)-1:
                ltemp=[m for m,x in enumerate(t2) if t1[i]<= x <=t1[i+1]]
                t_clust.append(ltemp)
            else:
                ltemp=[m for m,x in enumerate(t2) if t1[i]<= x]
                t_clust.append(ltemp) 
    if len(h1)>len(h2):
        print('You are increasing height resolution')
        for i in h1:
            x=np.argmin(np.abs(i-h2))
            h_clust.append(x)
    else:
        print('You are decreasing height resolution')     
        for i in range(len(h1)):
            if i!=len(h1)-1:
                ltemp=[m for m,x in enumerate(h2) if h1[i]<= x <=h1[i+1]]
                h_clust.append(ltemp)
            else:
                ltemp=[m for m,x in enumerate(h2) if h1[i]<= x]
                h_clust.append(ltemp) 
    print('--------End of clustering cycle--------')
    return t_clust,h_clust

In [5]:
A=8.07131
B=1730.63
C=233.426
l=2260000 #J/kg
a_cond=0.022 #W/m/K
a_water=5.5575 #W/m/K
D=0.399*10**-4 #m^2/s
Ra=287 #J/deg/kg
Rv=461 #J/deg/kg
qv=0.04 #kg/kg
rhow=1000 #kg/m3
rhoa=1.225 #kg/m3
g=9.8 #m2/s
cp=1000 #J/kg
C1=1.058 

In [6]:
def vaporpressure(temp):
    p=10**(A-(B)/(C+temp))
    p=p*133.322
    return p #in pascals

def constants(temp):
    vp=vaporpressure(temp-273.15)
    k=(1.52E-11)*(temp)**3-(4.8574E-8)*(temp)**2+1.0184E-04*(temp)-3.9333E-04
    F=(rhow*(l)**2/k/Rv/(temp)**2)+(rhow*Rv*(temp)/vp/D)
    A1=(g/Ra/temp*((l*Ra/(cp*Rv*temp))-1))
    A2=1/qv+((l)**2/cp/Rv/(temp)**2)
    return k,F,A1,A2

def C3(temp):
    k,F,A1,A2=constants(temp)
    a=C1*(F*A1/3)**0.75
    b=(3*rhoa/4/np.pi/rhow/A2)**0.5
    return a*b

def CDC_pinsky(temp,No,v,k):
    C=C3(temp)/10
    a=C**(2*k/(2+k))
    b=(No)**(2/(2+k))
    c=np.abs(v)**(3*k/(4+2*k))
    return a*b*c

In [7]:
def lwc_val(z,n,alpha=5,rhowa=1e-3):
    pf=np.pi/6*(math.factorial(3+alpha))/np.sqrt(math.factorial(6+alpha)*math.factorial(alpha))
    lw=pf*rhowa*np.sqrt(z*n)
    return lw*(10**3)

In [8]:
days=[]
for x in mp:
    days.append(x[-18:-10])

In [9]:
ll=generate_cdfs(days[0])  
k1,m1,r1,s1,e1,mp1,mwr1,mi1=ll
tk=k1['time'][:]
hk=k1['height'][:]
tr=r1['time_offset'][:]
hr=r1['height'][:]*1000
lwc_mb=m1['liquid_water_content'][:]
iwc=m1['ice_water_content'][:]
ref=10**(k1['reflectivity'][:]/10)
v=np.ma.filled(k1['mean_doppler_velocity'][:])
v2=v[:]
mmm=np.argwhere(v<0)
print('Checking velocity')
for mm in mmm:
    a,b=mm
    v2[a,b]=0.1
print('Checking temperature')
temp=r1['temperature'][:]+273.15
tc,hc=st_clustering(tk,hk,tr,hr)
temp_cl=[]
for i in tc:
    for j in hc:
        temp_cl.append(temp[i,j])
temp_cl=np.array(temp_cl)
temp_cl=np.reshape(temp_cl,(len(tk),len(hk)))
ts=np.ma.filled(s1['time'][:])
print('Checking CCn')
ccn=np.ma.filled(s1['N_CCN'][:])
tc2,dummy=st_clustering(tk,hk,ts,hr)
ccn_ts=[ccn[i] for i in tc2]
u=[[p]*len(hk) for p in ccn_ts]
ccn_cl=np.array(u)
tmi=mi1['time_offset'][:]
cb=np.ma.filled(mi1['first_cbh'][:])
cbm=mi1['first_cbh'].missing_value
tc2,dummy=st_clustering(tk,hk,tmi,hk)
print('Getting Cloud Base')
cbmod=[cb[f] for f in tc2]
cbmod=np.array(cbmod)
xx=np.where(cbmod<0)
for mm in xx:
    cbmod[mm]=0
cbmod=list(cbmod)
print('Getting Cloud Top')
cbup=cbmod[:]
for i,lx in enumerate(lwc_mb):
    po=np.argwhere(lx>0)
    if len(po)>0 and hk[po[-1]]<5000 and hk[po[-1]]>cbmod[i]:
        cbup[i]=np.ma.filled(hk[po[-1]])[0] 
print('Calculating CDC')
nt=CDC_pinsky(temp_cl,ccn_cl,v2,0.5)
print('Calculating LWC')
lwc_cal=lwc_val(ref,nt)
lwp_val=[]
lwp_mb=[]
tmw=mwr1['time']
lwp_mwr=mwr1['stat2_lwp']
print('Calculating LWP')
for i,x in enumerate(lwc_cal):
    if i%10000==0:
        print(i)
    hlow=np.argmin(np.abs(hk-cbmod[i]))
    hhigh=np.argmin(np.abs(hk-cbup[i]))
    hnew=hk[int(hlow):int(hhigh)]
    lnew=x[int(hlow):int(hhigh)]
    lwpp=np.trapz(lnew,hnew)
    lwp_val.append(lwpp)
for x in lwc_mb:
    lwpp=np.trapz(x,hk)
    lwp_mb.append(lwpp)
#Required variables: tk,hk,lwc_mb,v2,ref,ccn_cl,nt,temp_cl,cbmod,cbup,lwc_calc,lwp_mwr,lwp_cal,lwp_mb

Output has 6 files
File order is	1.KAZRARSCL	2.Microbase	3.Raman Lidar	4. Surface	5. Ext	6.Micropulse	 7.Mwrret	 8. Ceilometer
Checking velocity


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-73760e4d4a51>", line 15, in <module>
    a,b=mm
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 319, in

KeyboardInterrupt: 

In [ ]:
f=netCDF4.Dataset('cdfs_created/sample2.nc','w', format='NETCDF4')
f.createDimension('time',len(tk))
f.createDimension('height',len(hk))
f.createDimension('time_mwr',len(tmw[:]))
tim=f.createVariable('Time','f4','time')
ht=f.createVariable('Height','f4','height')
tmwr=f.createVariable('Time_mwr','f4','time_mwr')
lwcmb=f.createVariable('LWC_MB','f4',('time','height'))
vel=f.createVariable('Velocity','f4',('time','height'))
refl=f.createVariable('Reflectivity','f4',('time','height'))
ccn_c=f.createVariable('CCN','f4',('time','height'))
n=f.createVariable('CDC_pinsky','f4',('time','height'))
temp_c=f.createVariable('Temperature','f4',('time','height'))
lwpmr=f.createVariable('LWP_MR','f4','time_mwr')
lwpcal=f.createVariable('LWP_cal','f4','time')
lwpmb=f.createVariable('LWP_MB','f4','time')
cbm=f.createVariable('cloudbase','f4','time')
cbu=f.createVariable('cloudup','f4','time')
tim[:]=tk
ht[:]=hk
tmwr[:]=np.ma.filled(tmw[:])
lwcmb[:,:]=lwc_mb
vel[:,:]=v2
refl[:,:]=ref
ccn_c[:,:]=np.array(ccn_cl)
n[:,:]=nt
temp_c[:,:]=np.array(temp_cl)
d2=np.ma.filled(lwp_mwr[:])
lwpmr[:]=d2
lwpcal[:]=lwp_val
lwpmb[:]=lwp_mb
cbm[:]=cbmod
cbu[:]=cbup
f.description="Dataset generated from multiple data sources to estimate liquid water path"
today = datetime.today()
f.history="Created " + today.strftime("%d/%m/%y")
tim.units='s'
ht.units='m'
lwcmb.units='g m-3'
vel.units='ms-1'
refl.units='mm6m-3'
ccn_c.units='cm-3'
n.units='cm-3'
temp_c.units='K'
lwpmr.units='gm-2'
lwpcal.units='gm-2'
cbm.units='m'
cbu.units='m'
f.close()

In [13]:
m=Dataset('cdfs_created/sample2.nc')

In [10]:
def create_cdfs(d):
    ll=generate_cdfs(d)  
    k1,m1,r1,s1,e1,mp1,mwr1,mi1=ll
    tk=k1['time'][:]
    hk=k1['height'][:]
    tr=r1['time_offset'][:]
    hr=r1['height'][:]*1000
    lwc_mb=m1['liquid_water_content'][:]
    iwc=m1['ice_water_content'][:]
    ref=10**(k1['reflectivity'][:]/10)
    v=np.ma.filled(k1['mean_doppler_velocity'][:])
    v2=v[:]
    mmm=np.argwhere(v<0)
    print('Checking velocity')
    for mm in mmm:
        a,b=mm
        v2[a,b]=0.1
    print('Checking temperature')
    temp=r1['temperature'][:]+273.15
    tc,hc=st_clustering(tk,hk,tr,hr)
    temp_cl=[]
    for i in tc:
        for j in hc:
            temp_cl.append(temp[i,j])
    temp_cl=np.array(temp_cl)
    temp_cl=np.reshape(temp_cl,(len(tk),len(hk)))
    ts=np.ma.filled(s1['time'][:])
    print('Checking CCn')
    ccn=np.ma.filled(s1['N_CCN'][:])
    tc2,dummy=st_clustering(tk,hk,ts,hr)
    ccn_ts=[ccn[i] for i in tc2]
    u=[[p]*len(hk) for p in ccn_ts]
    ccn_cl=np.array(u)
    tmi=mi1['time_offset'][:]
    cb=np.ma.filled(mi1['first_cbh'][:])
    cbm=mi1['first_cbh'].missing_value
    tc2,dummy=st_clustering(tk,hk,tmi,hk)
    print('Getting Cloud Base')
    cbmod=[cb[f] for f in tc2]
    cbmod=np.array(cbmod)
    xx=np.where(cbmod<0)
    for mm in xx:
        cbmod[mm]=0
    cbmod=list(cbmod)
    print('Getting Cloud Top')
    cbup=cbmod[:]
    for i,lx in enumerate(lwc_mb):
        po=np.argwhere(lx>0)
        if len(po)>0 and hk[po[-1]]<5000 and hk[po[-1]]>cbmod[i]:
            cbup[i]=np.ma.filled(hk[po[-1]])[0] 
    print('Calculating CDC')
    nt=CDC_pinsky(temp_cl,ccn_cl,v2,0.5)
    print('Calculating LWC')
    lwc_cal=lwc_val(ref,nt)
    lwp_val=[]
    lwp_mb=[]
    tmw=mwr1['time']
    lwp_mwr=mwr1['stat2_lwp']
    print('Calculating LWP')
    for i,x in enumerate(lwc_cal):
        if i%10000==0:
            print(i)
        hlow=np.argmin(np.abs(hk-cbmod[i]))
        hhigh=np.argmin(np.abs(hk-cbup[i]))
        hnew=hk[int(hlow):int(hhigh)]
        lnew=x[int(hlow):int(hhigh)]
        lwpp=np.trapz(lnew,hnew)
        lwp_val.append(lwpp)
    for x in lwc_mb:
        lwpp=np.trapz(x,hk)
        lwp_mb.append(lwpp)
    #Required variables: tk,hk,lwc_mb,v2,ref,ccn_cl,nt,temp_cl,cbmod,cbup,lwc_calc,lwp_mwr,lwp_cal,lwp_mb
    f=netCDF4.Dataset(f'cdfs_created/{d}.nc','w', format='NETCDF4')
    f.createDimension('time',len(tk))
    f.createDimension('height',len(hk))
    f.createDimension('time_mwr',len(tmw[:]))
    tim=f.createVariable('Time','f4','time')
    ht=f.createVariable('Height','f4','height')
    tmwr=f.createVariable('Time_mwr','f4','time_mwr')
    lwcmb=f.createVariable('LWC_MB','f4',('time','height'))
    vel=f.createVariable('Velocity','f4',('time','height'))
    refl=f.createVariable('Reflectivity','f4',('time','height'))
    ccn_c=f.createVariable('CCN','f4',('time','height'))
    n=f.createVariable('CDC_pinsky','f4',('time','height'))
    temp_c=f.createVariable('Temperature','f4',('time','height'))
    lwpmr=f.createVariable('LWP_MR','f4','time_mwr')
    lwpcal=f.createVariable('LWP_cal','f4','time')
    lwpmb=f.createVariable('LWP_MB','f4','time')
    cbm=f.createVariable('cloudbase','f4','time')
    cbu=f.createVariable('cloudup','f4','time')
    tim[:]=tk
    ht[:]=hk
    tmwr[:]=np.ma.filled(tmw[:])
    lwcmb[:,:]=lwc_mb
    vel[:,:]=v2
    refl[:,:]=ref
    ccn_c[:,:]=np.array(ccn_cl)
    n[:,:]=nt
    temp_c[:,:]=np.array(temp_cl)
    d2=np.ma.filled(lwp_mwr[:])
    lwpmr[:]=d2
    lwpcal[:]=lwp_val
    lwpmb[:]=lwp_mb
    cbm[:]=cbmod
    cbu[:]=cbup
    f.description="Dataset generated from multiple data sources to estimate liquid water path"
    today = datetime.today()
    f.history="Created " + today.strftime("%d/%m/%y")
    tim.units='s'
    ht.units='m'
    lwcmb.units='g m-3'
    vel.units='ms-1'
    refl.units='mm6m-3'
    ccn_c.units='cm-3'
    n.units='cm-3'
    temp_c.units='K'
    lwpmr.units='gm-2'
    lwpcal.units='gm-2'
    cbm.units='m'
    cbu.units='m'
    f.close()
    return None

In [11]:
for d in days:
    create_cdfs(d)

Output has 6 files
File order is	1.KAZRARSCL	2.Microbase	3.Raman Lidar	4. Surface	5. Ext	6.Micropulse	 7.Mwrret	 8. Ceilometer
Checking velocity
Checking temperature
--------Start of clustering cycle--------
Changing time resolution from 144->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
Checking CCn
--------Start of clustering cycle--------
Changing time resolution from 1440->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
--------Start of clustering cycle--------
Changing time resolution from 5394->21600
Changing height resolution from 596->596
You are increasing time resolution
You are decreasing height resolution
--------End of clustering cycle--------
Getting Cloud Base
Getting Cloud Top
Calculating CDC
Calculating LWC
Calculating LWP
0
10000
20000


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: UserWarning: Warning: converting a masked element to nan.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: UserWarning: Warning: converting a masked element to nan.


Output has 6 files
File order is	1.KAZRARSCL	2.Microbase	3.Raman Lidar	4. Surface	5. Ext	6.Micropulse	 7.Mwrret	 8. Ceilometer
Checking velocity
Checking temperature
--------Start of clustering cycle--------
Changing time resolution from 144->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
Checking CCn
--------Start of clustering cycle--------
Changing time resolution from 1440->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
--------Start of clustering cycle--------
Changing time resolution from 5397->21600
Changing height resolution from 596->596
You are increasing time resolution
You are decreasing height resolution
--------End of clustering cycle--------
Getting Cloud Base
Getting Cloud Top
Calculating CDC
Calculating LWC
Calculating LWP
0
10000
20000
Output has 6

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Warning: converting a masked element to nan.


Checking CCn
--------Start of clustering cycle--------
Changing time resolution from 1440->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
--------Start of clustering cycle--------
Changing time resolution from 5397->21600
Changing height resolution from 596->596
You are increasing time resolution
You are decreasing height resolution
--------End of clustering cycle--------
Getting Cloud Base
Getting Cloud Top
Calculating CDC
Calculating LWC
Calculating LWP
0
10000
20000
Output has 6 files
File order is	1.KAZRARSCL	2.Microbase	3.Raman Lidar	4. Surface	5. Ext	6.Micropulse	 7.Mwrret	 8. Ceilometer
Checking velocity
Checking temperature
--------Start of clustering cycle--------
Changing time resolution from 144->21600
Changing height resolution from 198->596
You are increasing time resolution
You are increasing height resolution
--------End of clustering cycle--------
Checking CCn

In [7]:
l2=glob.glob('cdfs_created/*.nc')
l2

['cdfs_created\\20110505.nc',
 'cdfs_created\\20110513.nc',
 'cdfs_created\\20110514.nc',
 'cdfs_created\\20110519.nc',
 'cdfs_created\\20110527.nc',
 'cdfs_created\\20110529.nc']

In [11]:
for f in l2:
    dd=Dataset(f)
    print(dd)
    break

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: Dataset generated from multiple data sources to estimate liquid water path
    history: Created 29/09/20
    dimensions(sizes): time(21600), height(596), time_mwr(3515)
    variables(dimensions): float32 Time(time), float32 Height(height), float32 Time_mwr(time_mwr), float32 LWC_MB(time,height), float32 Velocity(time,height), float32 Reflectivity(time,height), float32 CCN(time,height), float32 CDC_pinsky(time,height), float32 Temperature(time,height), float32 LWP_MR(time_mwr), float32 LWP_cal(time), float32 LWP_MB(time), float32 cloudbase(time), float32 cloudup(time)
    groups: 
